<a href="https://colab.research.google.com/github/hrbolek/func2pipe/blob/news/notebooks/example01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install func2pipe

In [15]:
#import func2pipe.func2pipe as f2p
from func2pipe.func2pipe import createPipe, createSub, pipeit

## Unit Test

In [16]:
import unittest

class TestCase(unittest.TestCase):
  def __call__(self, *args, **kwargs):
    self.assertEqual(*args, **kwargs)
    return True

testEquality = TestCase()

## Simple Function

In [17]:
source = [{'value': 0}, {'value': 1}, {'value': 2}]
@pipeit()
def add(item, amount):
  return {**item, 'result': item['value'] + amount}

expectedResult = [{'value': 0, 'result': 3}, {'value': 1, 'result': 4},
  {'value': 2, 'result': 5}]

pipe = createPipe(
    add(amount = 2),
    add(amount = 3),
    list)

result = pipe(source)
print('Source:', source)
print('Result:', result)
print('Valid: ', testEquality(expectedResult, result)) 

Source: [{'value': 0}, {'value': 1}, {'value': 2}]
Result: [{'value': 0, 'result': 3}, {'value': 1, 'result': 4}, {'value': 2, 'result': 5}]
Valid:  True


## Function with Yield

In [18]:
source = [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]

@pipeit(with_yield = True)
def revealSubItem(item, itemName):
    for _ in item[itemName]:
        yield _

expectedResult = ['A', 'B', 'A', 'C', 'D', 'E']

pipe = createPipe(
    revealSubItem(itemName = 'value'),
    list)

result = pipe(source)
print('Source:', source)
print('Result:', result)
print('Valid: ', testEquality(expectedResult, result)) 

Source: [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]
Result: ['A', 'B', 'A', 'C', 'D', 'E']
Valid:  True


## Function with State

In [19]:
source = [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]

@pipeit(with_state = True)
def assignId(item, state = 0, idName = 'id'):
    return {**item, idName: state}, state + 1

expectedResult = [{'value': ['A', 'B'], 'ID': 0}, 
    {'value': ['A', 'C'], 'ID': 1}, {'value': ['D', 'E'], 'ID': 2}]

pipe = createPipe(
    assignId(idName = 'ID'),
    list)

result = pipe(source)
print('Source:', source)
print('Result:', result)
print('Valid: ', testEquality(expectedResult, result)) 

Source: [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]
Result: [{'value': ['A', 'B'], 'ID': 0}, {'value': ['A', 'C'], 'ID': 1}, {'value': ['D', 'E'], 'ID': 2}]
Valid:  True


## Function with Yield and with State

In [20]:
source = [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]

@pipeit(with_yield = True, with_state = True)
def assignIdToSubItem(item, state = -1, itemName = ''):
    for _ in item[itemName]:
        state = state + 1
        yield {itemName: _, 'id': state}, state

expectedResult = [{'value': 'A', 'id': 0}, {'value': 'B', 'id': 1}, 
  {'value': 'A', 'id': 2}, {'value': 'C', 'id': 3}, 
  {'value': 'D', 'id': 4}, {'value': 'E', 'id': 5}
  ]

pipe = createPipe(
    assignIdToSubItem(itemName = 'value'),
    list)

result = pipe(source)
print('Source:', source)
print('Result:', result)
print('Valid: ', testEquality(expectedResult, result)) 

Source: [{'value': ['A', 'B']}, {'value': ['A', 'C']}, {'value': ['D', 'E']}]
Result: [{'value': 'A', 'id': 0}, {'value': 'B', 'id': 1}, {'value': 'A', 'id': 2}, {'value': 'C', 'id': 3}, {'value': 'D', 'id': 4}, {'value': 'E', 'id': 5}]
Valid:  True


## Subpipe

In [21]:
source = [{'value': 0}, {'value': 1}, {'value': 2}]
@pipeit()
def plus(item, amount):
  return item + amount

@pipeit()
def selectIt(item, f):
  return f(item)

expectedResult = [
  {'value': 0, 'newvalue': 2}, 
  {'value': 1, 'newvalue': 3}, 
  {'value': 2, 'newvalue': 4}
  ]

pipe = createPipe(
    createSub(assign = lambda source, result: {**source, 'newvalue': result})(
      selectIt(f = lambda item: item['value']),
      plus(amount = 2),
    ),
    list)

result = pipe(source)
print('Source:', source)
print('Result:', result)
print('Valid: ', testEquality(expectedResult, result)) 

Source: [{'value': 0}, {'value': 1}, {'value': 2}]
Result: [{'value': 0, 'newvalue': 2}, {'value': 1, 'newvalue': 3}, {'value': 2, 'newvalue': 4}]
Valid:  True


In [22]:
#from func2pipe import createPipe, pipeit
#import func2pipe as fp

#print(fp.func2pipe)
@pipeit()
def r(item):
  return item + 2

operator = createPipe(
    r(),
    list
)

print(operator([0, 1, 2]))

[2, 3, 4]


## Standard Operations

### Select

In [ ]:
@pipeit()
def selectIt(item, f):
  return f(item)

### Filter

In [ ]:
@pipeit(with_yield=True)
def filterIt(item, f):
  if f(item):
    yield item